In [1]:
import pandas as pd
import numpy as np
import csv
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten

Using TensorFlow backend.
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zippy\anaconda3\envs\te

In [2]:
# This function should transform data into a usable format 
def featureProcess(path):
    data = pd.read_csv(path)
    #get audio data from vect1 to vect148
    #featureData = data.loc[:, "vect_1":"vect_148"]
    
    #get feature data from loudness to vect148
    featureData = data.loc[:, "loudness":"vect_148"]
    
    # normalization
    featureData=(featureData-featureData.mean())/featureData.std()
    
    return featureData.values.tolist()

In [3]:
# This function should transform data into a usable format 
def labelProcess(path):
    data = pd.read_csv(path)
    #get classes
    labels = data.loc[:,"genre"].values.tolist()
    
    return labels

In [4]:
def toOneHot(classes, labels):
    one_hot_labels = []

    for label in labels:
        one_hot_vector = [0] * len(classes)
        one_hot_vector[classes.index(label)] = 1

        one_hot_labels.append(one_hot_vector)
    return one_hot_labels

In [5]:
def CNN(trainData, trainLabel):
    
    num_classes = len(trainLabel[0])  # total classes to train
    model = Sequential()
    
    #all feature
    #model.add(Conv1D(128, 10,padding="same", input_shape=(148, 1), activation="relu", kernel_initializer="random_uniform"))
    
    #audio feature
    model.add(Conv1D(128, 10,padding="same", input_shape=(154, 1), activation="relu", kernel_initializer="random_uniform"))
    
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Conv1D(64, 10,padding="same", activation="relu", kernel_initializer="random_uniform"))
    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(50, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation="softmax", kernel_initializer="random_uniform"))
    
    
    #Define loss, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [6]:
def train(trainData, trainLabel, model):
    
    batch_size = 32  #the number of data to feed into model per batch
    num_epoch = 100  # go through your training data epoch times
    
    #callbacks stop training if val_loss is not improving.
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose = 1)]
    
    history = model.fit(trainData, trainLabel, validation_split=0.2, epochs = num_epoch, batch_size=batch_size, verbose=1, callbacks = callbacks)
                 
    return history, model

In [7]:
def evaluate(validData, validLabel, model):
    loss, accuracy = model.evaluate(validData, validLabel)
    return loss, accuracy

In [8]:
def predict(testData, model):
    result = model.predict_classes(testData, verbose=1)
    return result

DATA Preprocess

In [9]:
trainData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_features.csv")), axis=2)
validData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_features.csv")), axis=2)
testData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv")), axis=2)

In [10]:
trainLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_labels.csv")
validLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_labels.csv")
#get classes
classes = list(set(trainLabel))

oneHotTrain = np.array(toOneHot(classes, trainLabel))
oneHotvalid = np.array(toOneHot(classes, validLabel))

Training

In [11]:
model = CNN(trainData, oneHotTrain)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 154, 128)          1408      
_________________________________________________________________
dropout_1 (Dropout)          (None, 154, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 77, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 77, 64)            81984     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4928)              0         
_________________________________________________________________
dropout_2 (Dropout)  

In [12]:
history = train(trainData, oneHotTrain, model)
print("Finish training")

Instructions for updating:
Use tf.cast instead.
Train on 6142 samples, validate on 1536 samples
Epoch 1/100
6142/6142 [==============================] - 4s 711us/step - loss: 1.5264 - acc: 0.4302 - val_loss: 1.3065 - val_acc: 0.5234
Epoch 2/100
6142/6142 [==============================] - 1s 190us/step - loss: 1.3090 - acc: 0.5202 - val_loss: 1.2588 - val_acc: 0.5547
Epoch 3/100
6142/6142 [==============================] - 1s 190us/step - loss: 1.2110 - acc: 0.5583 - val_loss: 1.1784 - val_acc: 0.5729
Epoch 4/100
6142/6142 [==============================] - 1s 189us/step - loss: 1.1370 - acc: 0.5913 - val_loss: 1.1343 - val_acc: 0.5964
Epoch 5/100
6142/6142 [==============================] - 1s 186us/step - loss: 1.0625 - acc: 0.6110 - val_loss: 1.0948 - val_acc: 0.6172
Epoch 6/100
6142/6142 [==============================] - 1s 185us/step - loss: 1.0116 - acc: 0.6369 - val_loss: 1.0644 - val_acc: 0.6224
Epoch 7/100
6142/6142 [==============================] - 1s 183us/step - loss: 0.9

In [13]:
loss, accuracy = evaluate(validData, oneHotvalid, model)
print("loss, accuracy:", loss, accuracy)

450/450 [==============================] - 0s 157us/step
loss, accuracy: 1.6891232670678034 0.4822222222222222


Predict

In [ ]:
trackID = pd.read_csv(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv").loc[:, "trackID"].values.tolist()
prediction = predict(testData, model)

In [ ]:
results_name = []

for result in prediction:
    results_name.append(classes[result])

result = list(zip(trackID,results_name))

In [ ]:
#file = "audio_CNN_outPut.csv"
file = "meta+audio_CNN_outPut.csv"
with open(file, 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(["trackID", "genre"])
    writer.writerows(result)